In [1]:
import gzip
import json
import math
from collections import defaultdict
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Flatten, Concatenate, Dense, Multiply
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [2]:
path = 'goodreads_interactions_poetry.json.gz'

dataset = []

with gzip.open(path, 'rt', encoding='utf8') as file:
    # Process the file line by line
    for line in file:
        # Decode each line from JSON format
        json_content = json.loads(line)
        dataset.append(json_content)


In [5]:
usersPerItem = defaultdict(set)
itemsPerUser = defaultdict(set)
reviewsPerUser = defaultdict(list)
reviewsPerItem = defaultdict(list)
ratingsPerUser = defaultdict(list)
items = []
users = []
ratings = []

In [6]:
for d in dataset:
    user,item, rating = d['user_id'], d['book_id'], d['rating']
    usersPerItem[item].add(user)
    itemsPerUser[user].add(item)
    reviewsPerUser[user].append(d)
    ratingsPerUser[user].append(rating)
    reviewsPerItem[item].append(d)
    items.append(item)
    users.append(user)
    ratings.append(rating)

In [7]:
uniqueItems = list(set(items))
uniqueUsers = list(set(users))

In [9]:
# Extract unique user and item IDs from the subset using the predefined sets
subset_users = set()
subset_items = set()

for d in dataset:
    user, item, _ = d['user_id'], d['book_id'], d['rating']
    subset_users.add(user)
    subset_items.add(item)

all_users = list(subset_users.union(usersPerItem.keys()))
all_items = list(subset_items.union(itemsPerUser.keys()))

# Create user and item indices
user_index = {user: i for i, user in enumerate(all_users)}
item_index = {item: i for i, item in enumerate(all_items)}

# Convert reviews to numerical data
user_ids = [user_index[d['user_id']] for d in dataset]
item_ids = [item_index[d['book_id']] for d in dataset]
ratings = [d['rating'] for d in dataset]


In [ ]:
print(len(all_users))
print(len(all_items))

In [8]:
def create_ncf_model_with_gmf(num_users, num_items, embedding_size=64, hidden_units=(64, 32)):
    user_input = Input(shape=(1,), name='user_input')
    item_input = Input(shape=(1,), name='item_input')

    user_embedding = Embedding(input_dim=num_users, output_dim=embedding_size)(user_input)
    item_embedding = Embedding(input_dim=num_items, output_dim=embedding_size)(item_input)

    # GMF
    gmf_layer = Multiply()([user_embedding, item_embedding])

    user_flat = Flatten()(user_embedding)
    item_flat = Flatten()(item_embedding)
    gmf_flat = Flatten()(gmf_layer)

    # MLP
    mlp_output = Concatenate()([user_flat, item_flat])
    for units in hidden_units:
        mlp_output = Dense(units, activation='relu')(mlp_output)

    # CONCAT
    concat = Concatenate()([mlp_output, gmf_flat])

    output = Dense(1, activation='relu')(concat)

    model = Model(inputs=[user_input, item_input], outputs=output)
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

    return model



In [ ]:
# TRAIN AND EVAL
user_ids = np.array(user_ids)
item_ids = np.array(item_ids)
ratings = np.array(ratings)

ncf_model_with_gmf = create_ncf_model_with_gmf(len(all_users), len(all_items))

In [18]:

.fit([user_ids, item_ids], ratings, epochs=5, batch_sizncf_model_with_gmfe=1000, validation_split=0.2)

Epoch 1/5
2188/2188 [==============================] - 385s 176ms/step - loss: 2.9226 - mae: 1.3486 - val_loss: 4.3244 - val_mae: 1.8698
Epoch 2/5
2188/2188 [==============================] - 381s 174ms/step - loss: 2.3302 - mae: 1.1013 - val_loss: 4.3653 - val_mae: 1.8754
Epoch 3/5
2188/2188 [==============================] - 386s 176ms/step - loss: 1.9762 - mae: 0.9555 - val_loss: 4.4741 - val_mae: 1.8680
Epoch 4/5
2188/2188 [==============================] - 388s 177ms/step - loss: 1.5872 - mae: 0.7996 - val_loss: 4.5976 - val_mae: 1.8538
Epoch 5/5
2188/2188 [==============================] - 388s 177ms/step - loss: 1.2219 - mae: 0.6533 - val_loss: 4.7556 - val_mae: 1.8409


In [9]:
ncf_model = create_ncf_model_with_gmf(len(users), len(items))

In [10]:
ncf_model.fit([np.array(users), np.array(items)], np.array(ratings), epochs = 5, batch_size = 1000, validation_split = 0.2)

Epoch 1/5




UnimplementedError: Graph execution error:

Detected at node model/Cast defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "C:\Users\ics07\AppData\Roaming\Python\Python311\site-packages\ipykernel_launcher.py", line 17, in <module>

  File "C:\Users\ics07\AppData\Roaming\Python\Python311\site-packages\traitlets\config\application.py", line 1077, in launch_instance

  File "C:\Users\ics07\AppData\Roaming\Python\Python311\site-packages\ipykernel\kernelapp.py", line 739, in start

  File "C:\Users\ics07\AppData\Roaming\Python\Python311\site-packages\tornado\platform\asyncio.py", line 205, in start

  File "c:\Users\ics07\AppData\Local\Programs\Python\Python311\Lib\asyncio\base_events.py", line 607, in run_forever

  File "c:\Users\ics07\AppData\Local\Programs\Python\Python311\Lib\asyncio\base_events.py", line 1922, in _run_once

  File "c:\Users\ics07\AppData\Local\Programs\Python\Python311\L

In [19]:
ncf_model_with_gmf.save("ncf_with_gmf_model.h5")

c:\Users\ics07\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [20]:
print(user_ids[0])

126850


In [34]:
print(len(users))

377799


In [ ]:
ncf_model_with_gmf.predict([])

In [41]:
m = tf.keras.models.load_model('simple_model.h5')

In [44]:
print(item_ids[0:5])

[252711 271174  71923 233161  71923]


In [51]:
testUser = []
testItem = []
for i in range(5):
    testUser.append(1)
    testItem.append(item_ids[i])

In [54]:
testUser = np.array(testUser)
testItem = np.array(testItem)

In [55]:
m.predict([testUser, testItem])

1/1 [==============================] - 0s 106ms/step


array([[0.9758783],
       [0.9842483],
       [0.9919684],
       [0.9990616],
       [0.9919684]], dtype=float32)